In [1]:
import JuMP as jmp
import HiGHS as hgs

import CSV as csv
import DataFrames as dtf
import Plots as plt
import Statistics as st

In [19]:
Demand = Matrix(csv.read("HW2_demand_hourly.csv", dtf.DataFrame, normalizenames=true))

35×18 Matrix{Int64}:
  4  29  72   54   80  26  15  40   87  47  31  34   35  77  36  51  27   4
  7  21  50  118   62  54  27  77   45  42  46  73   53  86  45  37  26  19
 24   4  35   51   93  36  15  67   84  37  45  54   51  78  71  53  26  13
 12  19  38   76   47  21   6  62   48  44  38  19   43  55  70  40  39  13
 11   6  46   70  109  34  20  51   62  59  15  46   49  51  46  40  44  23
  8  15  54  109   53  35  28  51   69  66  32  48   52  60  38  40  11   6
  7  14  33   60   41  23  24  46   68  21  56  53   40  91  63  35  29  18
 11   9  64   29   53  44  23  40   52  32  43  64   43  57  28  47  20  16
  4  11  51  103   39  30  26  48   86  54  29  59   68  66  69  80  41  23
  4  14  49   66   41  29  29  55   43  66  22  48   76  89  44  63  40  12
  4   4  44   62   65  34  23  77  125  33  30  46   58  89  56  46  41  12
 17  11  39   70   61  36  17  63   97  48  18  55   44  60  70  39  37  14
  4   4  52   78   73  24  13  31   76  30  18  75   58  67  45  53

In [24]:
Ajt = Matrix(csv.read("Atj_input.csv", dtf.DataFrame, normalizenames=true))

99×18 Matrix{Int64}:
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0
 ⋮              ⋮              ⋮              ⋮     
 0  0  0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  0
 0  0  0  0  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  0  0  0
 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  0  0
 0  0  1

In [21]:
Cost = Matrix(csv.read("Cost_input.csv", dtf.DataFrame, normalizenames=true))

99×1 Matrix{Int64}:
  40
  40
  40
  40
  40
  40
  40
  40
  40
  40
  40
  40
  40
   ⋮
 400
 400
 440
 440
 440
 440
 480
 480
 480
 520
 520
 560

In [25]:
size(Ajt)

(99, 18)

In [26]:
model_a = jmp.Model(hgs.Optimizer)
max_day = 35 

#Xjd is the number of people working shift j on day d
jmp.@variable(model_a, X[1:size(Ajt)[1], 1:max_day] >= 0, Int) 

#for a given day/hour, number of workers working the shift >= demanded
for d in 1:max_day
    for t in 1:size(Ajt)[2]
        jmp.@constraint(model_a, sum(Ajt[j,t] * X[j,d] for j in 1:size(Ajt)[1]) >= Demand[d, t]) 
    end
end

#sum cost over all the shifts and days
total_cost=0
for d in 1:max_day
   for j in 1:size(Ajt)[1]
       total_cost += Cost[j] * X[j,d]
    end
end

jmp.@objective(model_a, Min, total_cost)
jmp.optimize!(model_a)

Presolving model
630 rows, 3465 cols, 28910 nonzeros
630 rows, 3465 cols, 28910 nonzeros
Objective function is integral with scale 0.05

Solving MIP model with:
   630 rows
   3465 cols (0 binary, 3465 integer, 0 implied int., 0 continuous)
   28910 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 T       0       0         0   0.00%   0               681380           100.00%        0      0      0       551     0.0s

Solving report
  Status            Optimal
  Primal bound      681380
  Dual bound        681380
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    681380 (objective)
                    0 (bound viol.)
       

In [29]:
X_output = dtf.DataFrame(jmp.value.(X), :auto)

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,20.0,0.0,5.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,18.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,8.0
4,0.0,60.0,4.0,30.0,24.0,55.0,19.0,0.0,60.0
5,26.0,4.0,46.0,1.0,63.0,0.0,0.0,9.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
9,21.0,0.0,0.0,0.0,17.0,0.0,22.0,0.0,11.0


Part c) See above for model implementation. Optimal cost is 681,380 for 35 days of demand. Staffing details for each shift for each day is in X_output above as well. 

In [41]:
model_b = jmp.Model(hgs.Optimizer)
max_day = 35 

#Xjd is the number of people working shift j on day d
jmp.@variable(model_b, X[1:size(Ajt)[1], 1:max_day] >= 0, Int) 

#for a given day/hour, number of workers working the shift >= demanded
for d in 1:max_day
    for t in 1:size(Ajt)[2]
        jmp.@constraint(model_b, sum(Ajt[j,t] * X[j,d] for j in 1:size(Ajt)[1]) >= Demand[d, t]) 
    end
end

#60 full time workers cannot work more than 2 days in a row
#full time shifts are j = 34 to j = 99
for d in 1:(max_day-2)
    jmp.@constraint(model_b, sum(X[j,d]+X[j,d+1]+X[j,d+2] for j in 34:99 ) <= 60*2)
end

#30 part time workers cannot work more than 2 days in a row
#part time shifts are j = 19 to j = 33
for d in 1:(max_day-2)
    jmp.@constraint(model_b, sum(X[j,d]+X[j,d+1]+X[j,d+2] for j in 19:33) <= 30*2)
end


#FT number of people for a given day <= 60 
for d in 1:(max_day)
    jmp.@constraint(model_b, sum(X[j,d] for j in 34:99 ) <= 60)
end


#PT number of people for a given day <= 30 
for d in 1:(max_day)
    jmp.@constraint(model_b, sum(X[j,d] for j in 19:33 ) <= 30)
end


#sum cost over all the shifts and days
total_cost=0
for d in 1:max_day
   for j in 1:size(Ajt)[1]
       total_cost += Cost[j] * X[j,d]
    end
end

jmp.@objective(model_b, Min, total_cost)
jmp.optimize!(model_b)

Presolving model
766 rows, 3465 cols, 39764 nonzeros
766 rows, 3465 cols, 39764 nonzeros
Objective function is integral with scale 0.05

Solving MIP model with:
   766 rows
   3465 cols (0 binary, 3465 integer, 0 implied int., 0 continuous)
   39764 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   836100          842700             0.78%        0      0      0      1060     0.2s
 C       0       0         0   0.00%   836120          842680             0.78%       36      6      0      1069     0.5s
 T       0       0         0   0.00%   836120          836120             0.00%       36      6      0      1069     0.5s

Solvin

In [42]:
X_output_b = dtf.DataFrame(jmp.value.(X), :auto)

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,7.0,20.0,12.0,5.0,8.0,0.0,2.0,4.0
2,15.0,21.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
3,32.0,0.0,20.0,6.0,20.0,31.0,10.0,35.0,37.0
4,14.0,68.0,36.0,44.0,44.0,86.0,37.0,0.0,77.0
5,44.0,8.0,78.0,15.0,83.0,25.0,18.0,19.0,13.0
6,0.0,0.0,21.0,0.0,8.0,7.0,0.0,10.0,4.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,21.0,38.0,52.0,16.0,9.0,19.0,11.0,17.0,22.0
9,59.0,6.0,52.0,2.0,20.0,37.0,33.0,11.0,60.0


In [38]:
model_c = jmp.Model(hgs.Optimizer)
max_day = 35 

#Xjd is the number of people working shift j on day d
jmp.@variable(model_c, X[1:size(Ajt)[1], 1:max_day] >= 0) 

#for a given day/hour, number of workers working the shift >= demanded
for d in 1:max_day
    for t in 1:size(Ajt)[2]
        jmp.@constraint(model_c, sum(Ajt[j,t] * X[j,d] for j in 1:size(Ajt)[1]) >= Demand[d, t]) 
    end
end

#60 full time workers cannot work more than 2 days in a row
#full time shifts are j = 34 to j = 99
for d in 1:(max_day-2)
    jmp.@constraint(model_c, sum(X[j,d]+X[j,d+1]+X[j,d+2] for j in 34:99 ) <= 60*2)
end

#30 part time workers cannot work more than 2 days in a row
#part time shifts are j = 19 to j = 33
for d in 1:(max_day-2)
    jmp.@constraint(model_c, sum(X[j,d]+X[j,d+1]+X[j,d+2] for j in 19:33) <= 30*2)
end


#FT number of people for a given day <= 60 
for d in 1:(max_day)
    jmp.@constraint(model_c, sum(X[j,d] for j in 34:99 ) <= 60)
end


#PT number of people for a given day <= 30 
for d in 1:(max_day)
    jmp.@constraint(model_c, sum(X[j,d] for j in 19:33 ) <= 30)
end


#sum cost over all the shifts and days
total_cost=0
for d in 1:max_day
   for j in 1:size(Ajt)[1]
       total_cost += Cost[j] * X[j,d]
    end
end

jmp.@objective(model_c, Min, total_cost)
jmp.optimize!(model_c)

Presolving model
766 rows, 3465 cols, 39764 nonzeros
766 rows, 3465 cols, 39764 nonzeros
Presolve : Reductions: rows 766(-0); columns 3465(-0); elements 39764(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 630(27554) 0s
       1008     8.3610000000e+05 Pr: 0(0); Du: 0(1.03739e-12) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1008
Objective value     :  8.3610000000e+05
HiGHS run time      :          0.03


In [40]:
X_output_c = dtf.DataFrame(jmp.value.(X), :auto)

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,7.0,20.0,12.0,5.0,8.0,0.0,2.0,4.0
2,15.0,21.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
3,32.0,0.0,20.0,6.0,19.5,31.0,10.0,35.0,37.5
4,14.0,68.0,36.0,44.0,43.5,86.0,37.0,0.0,77.0
5,44.0,8.0,78.0,15.0,82.5,25.0,18.0,18.0,13.0
6,0.0,0.0,21.0,0.0,7.5,7.0,0.0,9.0,4.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,21.0,37.5,52.0,16.0,8.5,19.5,11.0,17.0,22.0
9,59.0,5.5,52.5,2.0,19.5,37.5,33.0,11.0,60.0
